# Laboratory Exercise: Poker Hand Classification from 5 Cards

## Introduction

In this laboratory exercise, you will build a **multi-class classification model** that predicts the **poker hand category** given **five playing cards**. Each data sample represents a complete 5-card poker hand, and the task is to correctly classify it into one of the standard poker hand types (e.g. *Nothing in hand*, *One pair*, *Straight*, *Flush*, *Royal flush*, etc.).

You will implement a **full machine learning pipeline using PyTorch**, including data preprocessing, dataset definition, model building, training, evaluation, visualization, and final testing.

This exercise focuses on **structured categorical data**, **multi-class classification**, and correct usage of **CrossEntropyLoss**.

## Problem Definition

- **Task:** Multiclass classification
- **Target column:** `Hand`
- **Goal:** Predict what will the winning hand be

You will work with a provided dataset (`dataset.csv`) and implement a complete **machine learning training pipeline** using PyTorch.

## Tasks Overview

You are required to implement the following components:

1. **Data Preparation**
   - Load the `dataset.csv` file
   - Separate features from the target column `Hand`
   - Split the data into training, validation, and test sets
   - Apply any required preprocessing

2. **Dataset Class**
   - Implement an `PockerDataset` class compatible with PyTorch’s `DataLoader`

3. **Model Building**
   - Implement a `build_model` function that returns a neural network for multiclass classification

4. **Training and Evaluation**
   - Implement:
     - `train_one_epoch`
     - `evaluate`
     - `test`
   - Train the model for a fixed number of epochs
   - Track training loss, validation loss, and validation accuracy for each epoch

5. **Visualization**
   - Plot:
     - Training loss vs. epochs
     - Validation loss vs. epochs
     - Validation accuracy vs. epochs

6. **Testing and Reporting**
   - Evaluate the final model on the test dataset
   - Generate a **classification report** (precision, recall, F1-score)


## Model Comparison Requirement

You must design and train **two different model configurations**, for example:
- Different network depths or widths
- Different activation functions
- Different regularization strategies (e.g. dropout)

For **each model**, you must:
- Train it for the same number of epochs
- Plot training and validation metrics
- Evaluate it on the test set

## Dataset Description

You are given a CSV dataset with the following structure:

```text
Card 1,Card 2,Card 3,Card 4,Card 5,Hand
Jack Spades,King Spades,10 Spades,Queen Spades,Ace Spades,Royal flush
Queen Diamonds,Jack Diamonds,King Diamonds,10 Diamonds,Ace Diamonds,Royal flush
2 Heart,4 Heart,5 Heart,3 Heart,6 Heart,Straight flush
Ace Heart,Ace Spades,9 Diamonds,5 Heart,3 Spades,One pair
```

In [3]:
from typing import Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

In [4]:
from typing import Tuple
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer


def prepare_data(df: pd.DataFrame) -> Tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame,
    pd.DataFrame, pd.DataFrame, pd.DataFrame,
    ColumnTransformer
]:
    """
    Prepare the poker hands dataset for training and evaluation.

    The input DataFrame contains five cards describing a complete poker hand
    and a categorical target column "Hand" indicating the poker hand type
    (e.g. One pair, Straight, Flush, Royal flush).

    Steps (you MUST follow these steps):
    1. Split each card column ("Card 1" to "Card 5") into:
       - Rank (2–10, Jack, Queen, King, Ace)
       - Suit (Clubs, Diamonds, Hearts, Spades)
    2. Encode card features:
       - Ordinal-encode ranks using poker order
       - Label encode suits
    3. Separate features (X) and target (y), where the target column is "Hand".
    4. Label-encode the target labels into integer class indices.
    5. Use a ColumnTransformer to apply the encoders to all card features.
    6. Fit the preprocessor.
    7. Split the data in TWO stages (keep stratification):
       - First split into train and test:
            * test_size = 0.2
            * random_state = 42
            * stratify = y
       - Then split the training part into train and validation:
            * test_size = 0.2   (20% of the training set)
            * random_state = 42
            * stratify = y_train
    8. Return:
         X_train, X_val, X_test, y_train, y_val, y_test, preprocessor

    Notes:
    - The returned X arrays must be fully numeric.
    - The returned y arrays must contain integer class labels.
    - The data must be suitable for PyTorch multi-class classification.
    """
    df = df.copy()

    card_cols = ["Card 1", "Card 2", "Card 3", "Card 4", "Card 5"]
    for c in card_cols:
        if c not in df.columns:
            raise ValueError(f"Missing required column: {c}")
    if "Hand" not in df.columns:
        raise ValueError('Missing required target column: "Hand"')

    # --- 1) Split each card into Rank and Suit ---
    # Supports both singular and plural suits (e.g., "Heart" or "Hearts")
    suit_map = {
        "club": "Clubs", "clubs": "Clubs",
        "diamond": "Diamonds", "diamonds": "Diamonds",
        "heart": "Hearts", "hearts": "Hearts",
        "spade": "Spades", "spades": "Spades",
    }

    def parse_card(s: str) -> tuple[str, str]:
        if pd.isna(s):
            return ("", "")
        parts = str(s).strip().split()
        if len(parts) < 2:
            raise ValueError(f"Invalid card format: {s!r}")
        rank = " ".join(parts[:-1]).strip()  # usually one token, but safer
        suit_raw = parts[-1].strip().lower()
        suit = suit_map.get(suit_raw, None)
        if suit is None:
            raise ValueError(f"Unknown suit {parts[-1]!r} in card {s!r}")
        return (rank, suit)

    rank_cols, suit_cols = [], []
    for i, c in enumerate(card_cols, start=1):
        rcol = f"R{i}"
        scol = f"S{i}"
        parsed = df[c].apply(parse_card)
        df[rcol] = parsed.apply(lambda t: t[0])
        df[scol] = parsed.apply(lambda t: t[1])
        rank_cols.append(rcol)
        suit_cols.append(scol)

    # --- 3) Separate features/target ---
    X = df[rank_cols + suit_cols]
    y = df["Hand"]

    # --- 4) Label-encode target ---
    y_encoder = LabelEncoder()
    y_encoded = pd.Series(y_encoder.fit_transform(y), name="Hand")

    # --- 2 + 5) Encoders via ColumnTransformer ---
    # Poker order for ranks:
    rank_order = [["2", "3", "4", "5", "6", "7", "8", "9", "10",
                   "Jack", "Queen", "King", "Ace"]]
    suit_order = [["Clubs", "Diamonds", "Hearts", "Spades"]]

    preprocessor = ColumnTransformer(
        transformers=[
            ("rank_ord", OrdinalEncoder(categories=rank_order, handle_unknown="use_encoded_value", unknown_value=-1),
             rank_cols),
            ("suit_lbl", OrdinalEncoder(categories=suit_order, handle_unknown="use_encoded_value", unknown_value=-1),
             suit_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )

    # --- 6) Fit preprocessor ---
    preprocessor.fit(X)

    # Transform to numeric matrix; wrap as DataFrame for consistent return type
    X_numeric = pd.DataFrame(preprocessor.transform(X), columns=rank_cols + suit_cols, index=X.index)

    # --- 7) Two-stage split with stratification ---
    X_train, X_test, y_train, y_test = train_test_split(
        X_numeric, y_encoded,
        test_size=0.2,
        random_state=42,
        stratify=y_encoded
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )

    # --- 8) Return ---
    return X_train, X_val, X_test, y_train, y_val, y_test, preprocessor

In [5]:
df = pd.read_csv("dataset.csv")

In [6]:
df

,Card 1,Card 2,Card 3,Card 4,Card 5,Hand
0,Jack Spades,King Spades,10 Spades,Queen Spades,Ace Spades,Royal flush
1,Queen Diamonds,Jack Diamonds,King Diamonds,10 Diamonds,Ace Diamonds,Royal flush
2,10 Clubs,Jack Clubs,Ace Clubs,King Clubs,Queen Clubs,Royal flush
3,Ace Clubs,King Clubs,Queen Clubs,Jack Clubs,10 Clubs,Royal flush
4,2 Heart,4 Heart,5 Heart,3 Heart,6 Heart,Straight flush
...,...,...,...,...,...,...
25004,9 Diamonds,6 Spades,Jack Clubs,Queen Clubs,4 Spades,Nothing in hand
25005,Ace Clubs,10 Clubs,King Diamonds,4 Diamonds,10 Heart,One pair
25006,Ace Spades,10 Spades,4 Clubs,Ace Clubs,King Clubs,One pair
25007,Queen Spades,3 Clubs,10 Heart,Queen Heart,9 Clubs,One pair


In [7]:
X_train, X_val, X_test, y_train, y_val, y_test, preprocessor = prepare_data(df)

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).

In [42]:
class PockerDataset(Dataset):
    """
    A PyTorch Dataset for poker hand classification.

    Each sample consists of:
    - a numeric feature vector representing five playing cards
    - a multiclass label indicating the poker hand category

    Requirements:
    - __init__(self, X, y):
        * X: numpy array of numeric features
        * y: array-like of integer class labels
        * Store:
            - X as a float32 tensor
            - y as a long tensor (class indices)
    - __len__(self):
        * Return the number of samples
    - __getitem__(self, idx):
        * Return (X[idx], y[idx])
    """
    pass

In [43]:
def train_one_epoch(model: nn.Module,
                    train_loader: DataLoader,
                    criterion,
                    optimizer) -> float:
    """
    Train the model for ONE epoch on the training dataset.

    This is a multi-class classification task for poker hand prediction.

    Requirements:
    - Set the model to training mode using model.train()
    - Iterate over batches from train_loader
    - For each batch:
        * Compute model outputs (logits)
        * Compute the loss using CrossEntropyLoss
        * Zero the gradients
        * Perform backpropagation
        * Update model parameters using the optimizer
    - Accumulate the training loss over all batches
    - Return the average training loss as a float
      (total loss divided by the number of batches)
    """
    raise NotImplementedError()

In [44]:
def evaluate(model: nn.Module,
             val_loader: DataLoader,
             criterion: nn.Module) -> Tuple[float, float]:
    """
    Evaluate the model on the validation dataset.

    This is a multi-class poker hand classification task.

    Requirements:
    - Set the model to evaluation mode using model.eval()
    - Disable gradient computation using torch.no_grad()
    - Iterate over batches from val_loader
    - For each batch:
        * Compute model outputs (logits)
        * Compute and accumulate validation loss
        * Convert logits to predicted class labels using argmax
        * Collect predicted labels and true labels
    - Compute validation accuracy over the entire validation set
    - Return:
        - validation accuracy (float)
        - validation loss (float)
    """
    raise NotImplementedError()

In [45]:
def test(model: nn.Module,
         test_loader: DataLoader) -> tuple[Tensor, Tensor]:
    """
    Evaluate the trained model on the test dataset.

    This function performs inference for multi-class poker hand classification.

    Requirements:
    - Set the model to evaluation mode using model.eval()
    - Disable gradient computation using torch.no_grad()
    - Iterate over batches from test_loader
    - For each batch:
        * Compute model outputs (logits)
        * Convert logits to predicted class labels using argmax
        * Collect all predicted labels and true labels
    - Return:
        - Tensor of true labels (shape: N,)
        - Tensor of predicted labels (shape: N,)

    These outputs will be used to compute a classification report.
    """
    raise NotImplementedError()

In [46]:
def build_model_1(input_dim: int) -> nn.Module:
    """
    Build and return a PyTorch neural network for poker hand classification.

    Requirements:
    - Use nn.Sequential to define the model
    - The model must accept input vectors of size input_dim
    - The final layer must output logits for all poker hand classes
    - Do NOT apply Softmax in the final layer

    Note:
    - Use CrossEntropyLoss during training
    - You may include additional hidden layers, activations, or regularization
    """
    raise NotImplementedError()

In [47]:
def build_model_2(input_dim: int) -> nn.Module:
    """
    Build and return a second PyTorch neural network for poker hand classification.

    This model should differ from build_model_1 (e.g. depth, width, dropout).

    Requirements:
    - Use nn.Sequential to define the model
    - The model must accept input vectors of size input_dim
    - The final layer must output logits for all poker hand classes
    - Do NOT apply Softmax in the final layer

    Note:
    - Use CrossEntropyLoss during training
    - This model will be compared against build_model_1
    """
    raise NotImplementedError()

### Build the models

In [48]:
# Call the build functions

### Train model 1

In [49]:
epochs = 0
train_losses_1 = []
val_losses_1 = []
val_accuracies_1 = []

for epoch in range(epochs):

    # Call all required functions and store the computed metrics
    # (training loss, validation loss, and validation accuracy).

    train_loss =0
    val_acc = 0

    print(f"Epoch {epoch + 1}/{epochs} | Train loss: {train_loss:.4f} | Val acc: {val_acc:.4f}")

### Train model 2

In [50]:
epochs = 0
train_losses_2 = []
val_losses_2 = []
val_accuracies_2 = []

for epoch in range(epochs):

    # Call all required functions and store the computed metrics
    # (training loss, validation loss, and validation accuracy).

    train_loss =0
    val_acc = 0

    print(f"Epoch {epoch + 1}/{epochs} | Train loss: {train_loss:.4f} | Val acc: {val_acc:.4f}")

### Visualize

In [51]:
# Visualize training and validation loss on the same plot, and visualize the validation accuracy across epochs.

### Evaluate

In [52]:
# Evaluate on the test dataset